# **Importamos la librerias necesarias**

In [7]:
from _utils import pd, read_json, find_columns_with_nulls

# **Ahora cargamos los dataset necesarios**

In [8]:
df = pd.DataFrame(read_json("../../datasets/json/coins.json"))

# **Inspeccion de Datos**


Ahora le damos una vista general de los datos y se limpiaran los datos 

`df_coins`

In [9]:
df.head(5)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,60353.000000,1.185324e+12,1.0,1.264032e+12,3.940632e+10,64555.000,...,2024-03-14T07:10:36.635Z,67.810000,8.860604e+04,2013-07-06T00:00:00.000Z,None,2024-04-30T22:17:35.155Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...","[thorswap, dexalot, cube]"
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,3000.410000,3.646028e+11,2.0,3.646028e+11,1.924903e+10,3244.680,...,2021-11-10T14:24:19.604Z,0.432979,6.893516e+05,2015-10-20T00:00:00.000Z,"{'times': 65.47829036268304, 'currency': 'btc'...",2024-04-30T22:17:27.884Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...","[curve_ethereum, thorswap, openocean_finance, ..."
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.999586,1.105704e+11,3.0,1.105704e+11,6.098278e+10,1.003,...,2018-07-24T00:00:00.000Z,0.572521,7.443268e+01,2015-03-02T00:00:00.000Z,None,2024-04-30T22:15:26.224Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, huo...","[jupiter, raydium2, curve_ethereum, meteora, t..."
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,580.300000,8.890616e+10,4.0,8.890616e+10,2.073207e+09,602.910,...,2021-05-10T07:24:17.097Z,0.039818,1.453308e+06,2017-10-19T00:00:00.000Z,None,2024-04-30T22:17:48.955Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, okex, huo...","[uniswap_v3, thorswap, openocean_finance, trad..."
4,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,126.620000,5.638658e+10,5.0,7.247136e+10,3.975282e+09,139.460,...,2021-11-06T21:54:35.825Z,0.500801,2.498250e+04,2020-05-11T19:35:23.449Z,None,2024-04-30T22:17:25.085Z,Pixels Game,pixels-game,"[binance, bybit_spot, nami_exchange, upbit, ok...","[injective, cube]"


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8370 entries, 0 to 8369
Data columns (total 30 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                8370 non-null   object 
 1   symbol                            8370 non-null   object 
 2   name                              8370 non-null   object 
 3   image                             8370 non-null   object 
 4   current_price                     8367 non-null   float64
 5   market_cap                        8368 non-null   float64
 6   market_cap_rank                   3950 non-null   float64
 7   fully_diluted_valuation           7641 non-null   float64
 8   total_volume                      8368 non-null   float64
 9   high_24h                          8032 non-null   float64
 10  low_24h                           8032 non-null   float64
 11  price_change_24h                  8032 non-null   float64
 12  price_

Observamos la frecuencia de las exchanges que pertenece cada moneda

In [11]:
# Cantidad de exchanges centralizados (frecuencia)
df["exchanges_centralized"].apply(lambda x: len(x)).value_counts()[:10]

exchanges_centralized
0    3490
1    2361
2     808
3     394
4     256
5     123
6     104
8      74
7      70
9      56
Name: count, dtype: int64

In [12]:
# Cantidad de exchanges descentralizados (frecuencia)
df["exchanges_decentralized"].apply(lambda x: len(x)).value_counts()[:10]

exchanges_decentralized
1     3372
0     2749
2     1267
3      466
4      231
5      149
6       46
7       32
8       18
11       7
Name: count, dtype: int64

## Caso especial

Antes de realizar la limpieza de los datos, podemos observar los siguiente del `JSON` del dataset:
- La exchange a la cual pertenece es una version de esta pasada `uniswap_v2`, ya que actualmente esta la version 3.
- La criptomoneda no ha tenido movimiento las ultimas 24 horas.
- No tiene maximos ni minimos historicos.
  
Por lo cual podemos deducir que se debe a un error en el registro de la moneda o un problema con la version de la exchange de donde la API nos proporciono esta información 

```json
{
	"id": "party",
	"symbol": "party",
	"name": "PARTY",
	"image": "https://assets.coingecko.com/coins/images/33550/large/party-logo-200x200.png?1702397278",
	"current_price": null,
	"market_cap": null,
	"market_cap_rank": null,
	"fully_diluted_valuation": null,
	"total_volume": null,
	"high_24h": null,
	"low_24h": null,
	"price_change_24h": null,
	"price_change_percentage_24h": null,
	"market_cap_change_24h": null,
	"market_cap_change_percentage_24h": null,
	"circulating_supply": null,
	"total_supply": 420177600000000.0,
	"max_supply": 420177600000000.0,
	"ath": null,
	"ath_change_percentage": 0.0,
	"ath_date": null,
	"atl": null,
	"atl_change_percentage": 0.0,
	"atl_date": null,
	"roi": null,
	"last_updated": null,
	"category": "Pixels Game",
	"category_id": "pixels-game",
	"exchanges_centralized": [],
	"exchanges_decentralized": [
		"uniswap_v2"
	]
}
```

Este caso especial se tendra que eliminar del dataset 

In [13]:
# Eliminar la fila con id "party"
df = df[df["id"] != "party"]

In [14]:
# Validamos que no existe ese json
df[df["id"] == "party"]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized


# **Preprocesamiento de los Datos**

Eliminamos primero la columna `roi`, ya que no nos proporciona informacion relevante para el analisis

In [15]:
# Eliminamos la columna `roi`
df.drop(columns=["roi"], inplace=True)

Ahora verificamos que columnas tienen valores nulos

In [16]:
find_columns_with_nulls(df)

[('current_price', 1),
 ('market_cap_rank', 4418),
 ('fully_diluted_valuation', 727),
 ('high_24h', 336),
 ('low_24h', 336),
 ('price_change_24h', 336),
 ('price_change_percentage_24h', 336),
 ('market_cap_change_24h', 339),
 ('market_cap_change_percentage_24h', 339),
 ('total_supply', 633),
 ('max_supply', 2593)]

Eliminamos el unico registro que no tiene `current_price`

In [17]:
df = df[~df["current_price"].isnull()]

## Ahora revisamos la columna de `market_cap_rank`

In [18]:
find_columns_with_nulls(df)[0]

('market_cap_rank', 4417)

Observamos el maximo valor de `market_cap_rank`

In [19]:
df["market_cap_rank"].max() 

4385.0

In [20]:
df[3622:].head(4)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
3622,peruvian-national-football-team-fan-token,fpft,Peruvian National Football Team Fan Token,https://assets.coingecko.com/coins/images/2598...,0.006879,157326.0,3655.0,672422.0,73458.00,0.007113,...,-96.72469,2022-09-02T04:51:47.804Z,0.005258,31.23518,2023-07-22T09:50:54.170Z,2024-04-30T22:18:55.033Z,Pixels Game,pixels-game,[bitci],[]
3623,poseidon-2,psdn,Poseidon,https://assets.coingecko.com/coins/images/2531...,0.645963,157062.0,3656.0,645963.0,405.14,NaN,...,-98.60300,2022-04-30T12:09:26.157Z,0.343800,87.88901,2024-01-25T06:22:25.763Z,2024-04-26T08:42:50.230Z,Pixels Game,pixels-game,[],[balancer]
3624,eddaswap,edda,EDDASwap,https://assets.coingecko.com/coins/images/1436...,31.270000,156347.0,3657.0,156347.0,343.96,NaN,...,-99.47101,2021-03-15T20:24:28.894Z,23.000000,35.94726,2024-03-22T17:19:49.056Z,2024-04-29T21:45:19.055Z,Gaming (GameFi),gaming,[],[uniswap_v2]
3625,eddaswap,edda,EDDASwap,https://assets.coingecko.com/coins/images/1436...,31.270000,156347.0,3657.0,156347.0,343.96,NaN,...,-99.47101,2021-03-15T20:24:28.894Z,23.000000,35.94726,2024-03-22T17:19:49.056Z,2024-04-29T21:45:19.055Z,Pixels Game,pixels-game,[],[uniswap_v2]


Observamos que solo hay ranking hasta el numero `4333`, asi que rellenamos los nulos de esta columna con un `-1`

In [ ]:
df["market_cap_rank"].fillna(-1, inplace=True) # Rellenar con -1
df["market_cap_rank"] = df["market_cap_rank"].astype(int) # Convertir a entero

## Ahora revisamos la columna `fully_diluted_valuation`

In [22]:
find_columns_with_nulls(df)[0]

('fully_diluted_valuation', 726)

Observamos a detalle y notamos que:
- `fully_diluted_valuation` es igual a `total_supply * current_price` en ciertos casos
- Cuando `total_supply` es nulo, `fully_diluted_valuation` tambien es nulo


In [23]:
# Monedas con valor de `fully_diluted_valuation` igual al valor de `total_supply` por `current_price`
df[df["fully_diluted_valuation"] == df["total_supply"]*df["current_price"]].iloc[:3, 4:20]

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
2096,0.012197,6098525.0,1670,6098525.0,55186.00,0.012224,0.012187,0.000002,0.01893,-3522.818971,-0.05773,500000000.0,500000000.0,1.000000e+09,0.119948,-89.83135
2305,0.041805,4180508.0,1873,4180508.0,12.26,0.041932,0.041611,0.000103,0.24771,2107.110000,0.05043,100000000.0,100000000.0,1.000000e+08,0.089714,-53.40179
2839,0.048347,1450423.0,2475,4834743.0,1451.67,0.052356,0.047981,-0.003049,-5.93142,-91045.204540,-5.90640,30000000.0,100000000.0,1.000000e+08,0.146878,-67.08336


In [24]:
df[df["fully_diluted_valuation"].isnull()].iloc[:3, 7:20]

,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
62,NaN,59317483.0,131.070000,117.760000,-8.413910,-6.62987,-1.370793e+08,-5.99271,1.814782e+07,NaN,NaN,542.330,-78.15076
132,NaN,168093730.0,0.820115,0.742658,-0.040077,-5.01011,-4.277841e+07,-4.69011,1.148597e+09,NaN,NaN,22.710,-96.67488
156,NaN,6517660.0,0.979473,0.963034,-0.005005,-0.51380,-4.609538e+05,-0.06526,7.253320e+08,NaN,NaN,1.052,-7.70919


Entonces rellenaremos con el valor de `total_supply * current_price` cuando `fully_diluted_valuation` sea nulo y `total_supply` no sea nulo

In [ ]:
df["fully_diluted_valuation"].fillna(df["total_supply"]*df["current_price"], inplace=True)

In [26]:
find_columns_with_nulls(df)[0]

('fully_diluted_valuation', 632)

Ahora los restantes que no tenemos datos rellenaremos con -1

In [ ]:
df["fully_diluted_valuation"].fillna(-1, inplace=True) # Rellenar con -1

## Ahora verificamos la columna `high_24h`


In [28]:
find_columns_with_nulls(df)[0]

('high_24h', 335)

In [29]:
df[df["high_24h"].isnull()].iloc[:5, 0:20]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
880,ecoin-2,ecoin,Ecoin,https://assets.coingecko.com/coins/images/1084...,1.106400e-04,51020629.0,658,110644598.0,1.80,NaN,NaN,NaN,NaN,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,ethereum-meta,ethm,Ethereum Meta,https://assets.coingecko.com/coins/images/6586...,1.736100e-11,17378399.0,1141,17378399.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
1652,vecrv-dao-yvault,yve-crvdao,veCRV-DAO yVault,https://assets.coingecko.com/coins/images/1306...,6.471970e-01,13267302.0,1280,13267302.0,751.33,NaN,NaN,NaN,NaN,NaN,NaN,2.049962e+07,2.049962e+07,NaN,3.590000,-81.97719
2220,yuan-chain-coin,ycc,Yuan Chain Coin,https://assets.coingecko.com/coins/images/5328...,9.883300e-04,4903631.0,1784,9883336.0,180.50,NaN,NaN,NaN,NaN,NaN,NaN,4.961514e+09,1.000000e+10,NaN,0.040704,-97.57189
2296,micropets,pets,MicroPets [OLD],https://assets.coingecko.com/coins/images/1917...,4.856130e-07,4264537.0,1865,4856127.0,4.42,NaN,NaN,NaN,NaN,NaN,NaN,8.781766e+12,1.000000e+13,1.000000e+13,0.000036,-98.65228


Observamos que en otros registros el high_24h es igual al current_price, por lo cual rellenaremos los nulos con el valor de `current_price`

In [30]:
df[df["current_price"] == df["high_24h"]][:5]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
169,fasttoken,ftn,Fasttoken,https://assets.coingecko.com/coins/images/2847...,1.970000,612568756.0,130,1.731058e+09,22553521.00,1.970000,...,-0.15721,2024-04-30T00:56:25.348Z,0.398142,394.29382,2023-01-21T10:15:39.503Z,2024-04-30T22:17:17.015Z,Pixels Game,pixels-game,"[bitrue, whitebit, biconomy, xt, coinstore, fa...",[]
296,zetachain,zeta,ZetaChain,https://assets.coingecko.com/coins/images/2671...,1.300000,305167783.0,220,2.710093e+09,48820990.00,1.300000,...,-55.54770,2024-02-15T20:45:12.099Z,0.660431,92.06317,2024-02-01T05:41:28.321Z,2024-04-30T22:17:35.790Z,Pixels Game,pixels-game,"[bybit_spot, okex, huobi, bitget, bitrue, gdax...","[uniswap_v3, curve_ethereum, uniswap-bsc]"
2084,linda-2,linda,Linda,https://assets.coingecko.com/coins/images/3369...,0.000067,6299402.0,1654,6.671941e+06,96635.00,0.000067,...,-23.29603,2024-04-26T21:34:55.636Z,0.000004,1604.24099,2024-02-09T02:34:21.785Z,2024-04-30T22:18:01.660Z,Meme,meme-token,[],"[traderjoe, camelot]"
2085,linda-2,linda,Linda,https://assets.coingecko.com/coins/images/3369...,0.000067,6299402.0,1654,6.671941e+06,96635.00,0.000067,...,-23.29603,2024-04-26T21:34:55.636Z,0.000004,1604.24099,2024-02-09T02:34:21.785Z,2024-04-30T22:18:01.660Z,Pixels Game,pixels-game,[],"[traderjoe, camelot]"
2192,molecules-of-korolchuk-ip-nft,vita-fast,Molecules of Korolchuk IP-NFT,https://assets.coingecko.com/coins/images/3305...,5.150000,5147918.0,1757,5.147918e+06,333.79,5.150000,...,-68.32088,2023-12-08T01:05:57.617Z,2.390000,115.22388,2023-11-30T00:25:40.906Z,2024-04-30T22:16:59.447Z,Pixels Game,pixels-game,[],[uniswap_v3]


In [ ]:
df["high_24h"].fillna(df["current_price"], inplace=True)

## Verificamos la columna `low_24h`

In [32]:
find_columns_with_nulls(df)[0]

('low_24h', 335)

In [33]:
df[df["low_24h"].isnull()].iloc[:5, 0:20]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
880,ecoin-2,ecoin,Ecoin,https://assets.coingecko.com/coins/images/1084...,1.106400e-04,51020629.0,658,110644598.0,1.80,1.106400e-04,NaN,NaN,NaN,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,ethereum-meta,ethm,Ethereum Meta,https://assets.coingecko.com/coins/images/6586...,1.736100e-11,17378399.0,1141,17378399.0,0.00,1.736100e-11,NaN,NaN,NaN,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
1652,vecrv-dao-yvault,yve-crvdao,veCRV-DAO yVault,https://assets.coingecko.com/coins/images/1306...,6.471970e-01,13267302.0,1280,13267302.0,751.33,6.471970e-01,NaN,NaN,NaN,NaN,NaN,2.049962e+07,2.049962e+07,NaN,3.590000,-81.97719
2220,yuan-chain-coin,ycc,Yuan Chain Coin,https://assets.coingecko.com/coins/images/5328...,9.883300e-04,4903631.0,1784,9883336.0,180.50,9.883300e-04,NaN,NaN,NaN,NaN,NaN,4.961514e+09,1.000000e+10,NaN,0.040704,-97.57189
2296,micropets,pets,MicroPets [OLD],https://assets.coingecko.com/coins/images/1917...,4.856130e-07,4264537.0,1865,4856127.0,4.42,4.856130e-07,NaN,NaN,NaN,NaN,NaN,8.781766e+12,1.000000e+13,1.000000e+13,0.000036,-98.65228


Observamos que en otros registros el low_24h es igual al current_price, por lo cual rellenaremos los nulos con el valor de `current_price`


In [34]:
df[df["current_price"] == df["low_24h"]][:5]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,last_updated,category,category_id,exchanges_centralized,exchanges_decentralized
316,deso,deso,Decentralized Social,https://assets.coingecko.com/coins/images/1631...,24.41,257625035.0,236,264381996.0,9835176.00,26.31,...,-87.70420,2021-06-19T20:53:25.052Z,5.330000,358.44374,2022-09-01T18:05:41.566Z,2024-04-30T22:17:37.281Z,Pixels Game,pixels-game,"[huobi, bitrue, gdax, bitmax, coinex]",[]
635,ergo,erg,Ergo,https://assets.coingecko.com/coins/images/2484...,1.26,94971632.0,468,123849092.0,960641.00,1.36,...,-93.24842,2021-09-03T00:00:00.000Z,0.094191,1242.01421,2020-03-13T02:24:59.299Z,2024-04-30T22:18:32.739Z,Pixels Game,pixels-game,"[probit, hitbtc, hitbtc, bitcoin_com, bitcoin_...",[]
930,whiteheart,white,Whiteheart,https://assets.coingecko.com/coins/images/1348...,5062.04,44991411.0,699,44991411.0,1106.28,5537.50,...,-20.82174,2024-03-05T06:01:09.997Z,57.100000,8765.21682,2022-06-18T21:11:01.884Z,2024-04-30T16:25:24.496Z,Pixels Game,pixels-game,[],[uniswap_v3]
959,polytrade,trade,Polytrade,https://assets.coingecko.com/coins/images/1641...,1.16,42301181.0,723,116549471.0,672696.00,1.32,...,-61.42857,2024-04-01T09:09:58.768Z,0.051084,2191.38983,2022-07-03T19:48:00.852Z,2024-04-30T22:18:25.328Z,Pixels Game,pixels-game,"[gate, mxc, bitfinex]","[uniswap_v2, uniswap_v3_polygon_pos, camelot-v3]"
960,polytrade,trade,Polytrade,https://assets.coingecko.com/coins/images/1641...,1.16,42301181.0,723,116549471.0,673483.00,1.32,...,-61.42857,2024-04-01T09:09:58.768Z,0.051084,2191.38983,2022-07-03T19:48:00.852Z,2024-04-30T22:20:32.207Z,Real World Assets (RWA),real-world-assets-rwa,"[gate, mxc, bitfinex]","[uniswap_v2, uniswap_v3_polygon_pos, camelot-v3]"


In [ ]:
df["low_24h"].fillna(df["current_price"], inplace=True)

## Verificamos la columna `price_change_24h` y `price_change_percentage_24h`

In [36]:
find_columns_with_nulls(df)[0]

('price_change_24h', 335)

In [37]:
df[df["price_change_24h"].isnull()].iloc[:5, 0:20]

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
880,ecoin-2,ecoin,Ecoin,https://assets.coingecko.com/coins/images/1084...,1.106400e-04,51020629.0,658,110644598.0,1.80,1.106400e-04,1.106400e-04,NaN,NaN,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,ethereum-meta,ethm,Ethereum Meta,https://assets.coingecko.com/coins/images/6586...,1.736100e-11,17378399.0,1141,17378399.0,0.00,1.736100e-11,1.736100e-11,NaN,NaN,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
1652,vecrv-dao-yvault,yve-crvdao,veCRV-DAO yVault,https://assets.coingecko.com/coins/images/1306...,6.471970e-01,13267302.0,1280,13267302.0,751.33,6.471970e-01,6.471970e-01,NaN,NaN,NaN,NaN,2.049962e+07,2.049962e+07,NaN,3.590000,-81.97719
2220,yuan-chain-coin,ycc,Yuan Chain Coin,https://assets.coingecko.com/coins/images/5328...,9.883300e-04,4903631.0,1784,9883336.0,180.50,9.883300e-04,9.883300e-04,NaN,NaN,NaN,NaN,4.961514e+09,1.000000e+10,NaN,0.040704,-97.57189
2296,micropets,pets,MicroPets [OLD],https://assets.coingecko.com/coins/images/1917...,4.856130e-07,4264537.0,1865,4856127.0,4.42,4.856130e-07,4.856130e-07,NaN,NaN,NaN,NaN,8.781766e+12,1.000000e+13,1.000000e+13,0.000036,-98.65228


In [ ]:
df["price_change_24h"].fillna(0, inplace=True)
df["price_change_percentage_24h"].fillna(0, inplace=True)

## Verificamos la columna `market_cap_change_24h` y `market_cap_change_percentage_24h`


In [39]:
find_columns_with_nulls(df)[0]

('market_cap_change_24h', 338)

In [40]:
find_columns_with_nulls(df)[1]

('market_cap_change_percentage_24h', 338)

In [41]:
df[df["market_cap_change_24h"].isnull()].iloc[:5, 5:20]

,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
880,51020629.0,658,110644598.0,1.80,1.106400e-04,1.106400e-04,0.000000,0.00000,NaN,NaN,4.611217e+11,1.000000e+12,NaN,0.012765,-99.13320
1469,17378399.0,1141,17378399.0,0.00,1.736100e-11,1.736100e-11,0.000000,0.00000,NaN,NaN,1.000991e+18,1.000991e+18,NaN,0.041412,-100.00000
1652,13267302.0,1280,13267302.0,751.33,6.471970e-01,6.471970e-01,0.000000,0.00000,NaN,NaN,2.049962e+07,2.049962e+07,NaN,3.590000,-81.97719
1707,11673014.0,1331,11673014.0,38109.00,1.214382e+04,8.100340e+03,3557.000000,41.92895,NaN,NaN,9.694870e+02,9.694870e+02,9.694870e+02,166135.000000,-92.75263
2219,4908196.0,1783,4909445.0,23244.00,3.577000e-05,2.664000e-05,-0.000009,-25.41896,NaN,NaN,1.840200e+11,1.840668e+11,1.844700e+11,0.145166,-99.98163


In [ ]:
df["market_cap_change_24h"].fillna(0, inplace=True)
df["market_cap_change_percentage_24h"].fillna(0, inplace=True)

## Verificamos la columna `total_supply`


In [43]:
find_columns_with_nulls(df)[0]

('total_supply', 632)

In [44]:
df[df["total_supply"].isnull()].iloc[:5, 15:20]

,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
62,1.814782e+07,NaN,NaN,542.330000,-78.15076
132,1.148597e+09,NaN,NaN,22.710000,-96.67488
156,7.253320e+08,NaN,NaN,1.052000,-7.70919
328,7.550673e+10,NaN,NaN,0.046712,-93.30827
331,1.448188e+08,NaN,NaN,34.920000,-95.43014


Supondremos que el `circulating_supply` es el mismo que `total_supply`, ya que en algunas monedas es comun esto

In [45]:
df[df["circulating_supply"] != df["total_supply"]].iloc[:5, 15:20]

,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
0,1.969238e+07,2.100000e+07,2.100000e+07,73738.00,-18.42641
4,4.474404e+08,5.750767e+08,NaN,259.96,-51.67955
5,3.309605e+10,3.321095e+10,NaN,1.17,-14.69897
6,9.337744e+06,9.338133e+06,NaN,4829.57,-38.23986
7,5.523591e+10,9.998765e+10,1.000000e+11,3.40,-85.37527


In [ ]:
df["total_supply"].fillna(df["circulating_supply"], inplace=True)

## Verificamos la columna `max_supply`

In [47]:
find_columns_with_nulls(df)[0]

('max_supply', 2592)

In [48]:
df[df["max_supply"].isnull()].iloc[:5, 15:20]

,circulating_supply,total_supply,max_supply,ath,ath_change_percentage
1,1.220564e+08,1.220564e+08,NaN,4878.26,-38.80647
2,1.106432e+11,1.106432e+11,NaN,1.32,-24.52065
4,4.474404e+08,5.750767e+08,NaN,259.96,-51.67955
5,3.309605e+10,3.321095e+10,NaN,1.17,-14.69897
6,9.337744e+06,9.338133e+06,NaN,4829.57,-38.23986


Que `max_supply` sa nulo puede significar que la moneda no tiene limite de emision

In [ ]:
df["max_supply"].fillna(-1, inplace=True)

Una vez limpio los datos podemos observar los datos estadisticos de estos

In [50]:
df.describe()

,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,atl,atl_change_percentage
count,8.367000e+03,8.367000e+03,8367.000000,8.367000e+03,8.367000e+03,8.367000e+03,8.367000e+03,8.367000e+03,8367.000000,8.367000e+03,8367.000000,8.367000e+03,8.367000e+03,8.367000e+03,8.367000e+03,8367.000000,8367.000000,8.367000e+03
mean,3.889064e+02,2.947104e+08,822.439584,3.186563e+43,2.346463e+07,4.327446e+02,3.617818e+02,-1.477985e+01,-3.756951,-1.453022e+07,-1.318639,2.270173e+19,2.390343e+72,2.390343e+72,1.407910e+04,-80.926013,179.581742,2.186693e+42
std,1.701896e+04,1.368189e+10,1187.329771,2.060945e+45,8.376435e+08,1.937869e+04,1.625592e+04,2.192486e+03,44.718508,6.757117e+08,44.273312,1.468219e+21,1.545981e+74,1.545981e+74,7.769487e+05,23.433881,10798.515211,1.894682e+44
min,0.000000e+00,0.000000e+00,-1.000000,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.703741e+05,-91.561320,-5.707336e+10,-100.000000,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,-100.000000,-270.577376,-1.283882e+02
25%,3.923450e-04,0.000000e+00,-1.000000,2.538620e+05,2.071440e+03,4.289950e-04,3.603050e-04,-5.164625e-03,-8.515890,-2.193716e+05,-4.865295,0.000000e+00,4.501016e+07,-1.000000e+00,8.258930e-03,-98.238695,0.000090,2.408953e+01
50%,9.169340e-03,0.000000e+00,-1.000000,2.849204e+06,5.101800e+04,1.031648e-02,8.680160e-03,-1.450955e-04,-4.922510,0.000000e+00,0.000000,0.000000e+00,6.500000e+08,1.000000e+08,2.418480e-01,-90.936750,0.002209,1.259296e+02
75%,1.793355e-01,6.155237e+06,1451.500000,2.947488e+07,4.597280e+05,1.988300e-01,1.706510e-01,-2.311598e-08,-0.660090,0.000000e+00,0.000000,2.000000e+08,3.000000e+09,1.000000e+09,2.450000e+00,-71.576460,0.041461,5.786939e+02
max,1.502316e+06,1.185324e+12,4385.000000,1.333099e+47,6.098278e+10,1.716157e+06,1.451128e+06,1.038180e+05,2506.164860,1.507412e+08,2540.430930,9.497004e+22,1.000000e+76,1.000000e+76,6.322948e+07,1.543600,974955.000000,1.730252e+46


Guardamos el dataset limpio


In [51]:
df.to_csv("../../datasets/csv/coins_clean.csv", index=False)

In [52]:
df.to_json("../../datasets/json/coins_clean.json", orient="records")